# 4. Basic Operations
http://docs.sympy.org/latest/tutorial/basic_operations.html

In [6]:
from sympy import *
init_printing()

x, y, z = symbols('x y z')

## Substitution
One of the most common things you might want to do with a mathematical expression is substitution. Substitution replaces all instances of something in an expression with something else. It is done using the ***subs*** method. For example

In [7]:
expr = cos(x) + 1
expr.subs(x, y)

Substitution is usually done for one of two reasons:

- **Evaluating an expression at a point**. For example, if our expression is cos(x) + 1 and we want to evaluate it at the point x = 0, so that we get cos(0) + 1, which is 2.

In [8]:
expr.subs(x, 0)

- **Replacing a subexpression with another subexpression**. There are two reasons we might want to do this. 
  - The first is if we are trying to build an expression that has some symmetry, such as $x^{x^{x^x}}$. To build this, we might start with x**y, and replace y with x**y. We would then get x**(x**y). If we replaced y in this new expression with x**x, we would get x**(x**(x**x)), the desired expression.

In [9]:
expr = x**y
expr

In [10]:
expr = expr.subs(y, x**y)
expr

In [11]:
expr = expr.subs(y, x**x)
expr

  - The second is if we want to perform ***a very controlled simplification***, or perhaps a simplification that SymPy is otherwise unable to do. For example, say we have $\sin{(2x)} + \cos{(2x)}$, and we want to replace sin⁡(2x) with 2sin⁡(x)cos⁡(x). As we will learn later, the function ***expand_trig*** does this. However, this function will also expand cos(2x)cos⁡(2x), which we may not want. While there are ways to perform such precise simplification, and we will learn some of them in the advanced expression manipulation section, an easy way is to just replace sin(2x) with 2sin(x)cos(x). 

In [12]:
expr = sin(2*x) + cos(2*x)
expr

In [13]:
expand_trig(expr)

In [14]:
expr = expr.subs(sin(2*x), 2*sin(x)*cos(x))
expr

There are two important things to note about subs. First, ***it returns a new expression***. SymPy objects are immutable. That means that subs does not modify it in-place. For example

In [15]:
expr = cos(x)
expr.subs(x, 0)

In [16]:
expr

In [17]:
x

#### Quick Tip
SymPy expressions are immutable. No function will change them in-place.

Here, we see that performing expr.subs(x, 0) leaves expr unchanged. In fact, since SymPy expressions are immutable, no function will change them in-place. ***All functions will return new expressions***.

To perform multiple substitutions at once, pass a list of (old, new) pairs to subs.

In [18]:
expr = x**3 + 4*x*y - z
expr.subs([(x, 2), (y, 4), (z, 0)])

It is often useful to combine this with a list comprehension to do a large set of similar replacements all at once. For example, say we had $x^4 - 4x^3 + 4x^2 -
2x + 3$ and we wanted to replace all instances of xx that have an even power with y, to get $y^4 - 4x^3 + 4y^2 - 2x + 3$.

In [19]:
expr = x**4 - 4*x**3 + 4*x**2 - 2*x + 3
replacements = [(x**i, y**i) for i in range(5) if i % 2 == 0]
expr.subs(replacements)

## Converting Strings to SymPy Expressions
The ***sympify*** function (that’s sympify, not to be confused with simplify) can be used to convert strings into SymPy expressions.

For example

In [20]:
str_expr = 'x**2 + 3*x - 1/2'
expr = sympify(str_expr)
expr

In [21]:
expr.subs(x, 2)

#### Warning
sympify uses eval. Don’t use it on unsanitized input.

## evalf
To evaluate a ***numerical expression*** into a ***floating point number***, use evalf.

In [22]:
expr = sqrt(8)
expr

In [23]:
expr.evalf()

SymPy can evaluate floating point expressions to arbitrary precision. ***By default, 15 digits***of precision are used, but you can pass any number as the argument to ***evalf***. Let’s compute the first 100 digits of ππ.

In [24]:
pi.evalf(100)

To numerically evaluate an expression with a Symbol at a point, we might use subs followed by evalf, but it is more efficient and numerically stable to pass the substitution to ***evalf*** using the ***subs*** flag, which takes a dictionary of Symbol: point pairs.

In [25]:
expr = cos(2*x)
expr.evalf(subs = {x: 2.4})

Sometimes there are roundoff errors smaller than the desired precision that remain after an expression is evaluated. Such numbers can be removed at the user’s discretion by setting the ***chop*** flag to True.

In [26]:
one = cos(1)**2 + sin(1)**2
one

In [27]:
(one - 1).evalf()

In [28]:
(one - 1).evalf(chop = True)

## lambdify
subs and evalf are good if you want to do simple evaluation, but if you intend to evaluate an expression at many points, there are more efficient ways. For example, if you wanted to evaluate an expression at a thousand points, using SymPy would be far slower than it needs to be, especially if you only care about machine precision. Instead, you should use libraries like NumPy and SciPy.

The easiest way to ***convert a SymPy expression to an expression that can be numerically evaluated*** is to use the ***lambdify function***. lambdify acts like a lambda function, except it converts the SymPy names to the names of the given numerical library, usually NumPy. For example

In [29]:
expr = sin(x)
f = lambdify(x, expr, 'numpy')

a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
f(a)

array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])

You can use other libraries than NumPy. For example, to use the standard library math module, use "math".

In [30]:
f = lambdify(x, expr, 'math')
f(0.1)

To use lambdify with numerical libraries that it does not know about, pass a dictionary of ***sympy_name:numerical_function*** pairs. For example

In [31]:
def mysin(x):
    return x

f = lambdify(x, expr, {'sin': mysin})
f(0.1)